# double code



- v = drift rate
- a = boundary separation
- Ter = non-decision time
- lamda = leakage rate
- beta = amount of inhibition
- al_n = number of alternative responses

# RDM

**parameters we need here:**

- v = drift rate
- a = boundary separation
- Ter = non-decision time


In [ ]:
# import all needed packages

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import ipynbname
import bayesflow as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from pathlib import Path

ModuleNotFoundError: No module named 'bayesflow'

In [ ]:
# defing the context

# selecting possible amounts of n -> for our example we could use a fixed value
# of n because all trials are 10000

def context(n=None):
    if n is None:
        n = np.random.randint(2, 251)
    return dict(n=n)

In [ ]:
# defining the priors

def prior(drift=None, boundary=None, starting_point=None, ndt=None):
    if drift is None:
        drift=np.random.dirichlet([2, 2])
        drift=np.random.gamma(shape=5, scale=0.5) * drift
    if boundary is None:                    #give it a feasible boundary
        boundary=np.random.gamma(shape=3, scale=1)
    if starting_point is None:
        starting_point=np.random.gamma(shape=7, scale=0.1)
    if ndt is None:
        ndt=np.random.exponential(15)
    else:
        ndt=ndt.item()
    return dict(drift=drift, boundary=boundary, starting_point=starting_point, ndt=ndt)


In [ ]:

@nb.jit(nopython=True, cache=True)
def trial(drift, starting_point, boundary, ndt, max_t, max_drt=0.25, s=1, dt=None):
    drift = np.asarray(drift, dtype=np.float64)  # Convert before passing to JIT
    response = -1
    rt = -1

    if dt is None:
        dt = max_t / 10_000.0  # Ensure float division

    t = 0
    start = float(starting_point)  # Ensure float type
    evidence = np.random.uniform(0, start, size=len(drift))

    boundary += start  # Adjust boundary based on start
    dr = False  # Initialize double response

    # Initial evidence accumulation
    while np.all(evidence < boundary) and t < max_t:
        for resp in range(len(drift)):  # Normal loop (prange if parallel)
            evidence[resp] += dt * drift[resp] + np.random.normal(0, np.sqrt(s**2 * dt))
        t += dt

    rt = t + ndt
    drt = 0
    response_arr = np.where(evidence > boundary)[0]  # Avoid tuple issue

    if response_arr.size > 0:
        response = response_arr[0]  # Take first element
    else:
        response = -1  # Default

    # Double response accumulation
    while drt < max_drt and not dr:
        for resp in range(len(drift)):
            if response != -1 and resp != response:
                evidence[resp] += dt * drift[resp] + np.random.normal(0, s) * np.sqrt(dt)
                if evidence[resp] >= boundary:
                    dr = True
                    break
        drt += dt  # Only increase while dr is False

    return rt, response, dr, drt

In [ ]:
# generate data for n trials
# keep the data.shape always to max_n
# the rest is filled with 0s


# insert dt and drt as a liklyhood

def likelihood(n, drift, boundary, starting_point, ndt, max_t=3.0, dt=0.02, max_n=300):
    rt = np.zeros(max_n)
    response = np.zeros(max_n)
    observed = np.zeros(max_n)
    dr=np.zeros(max_n)
    drt=np.zeros(max_n)

    for i in range(n):
        rt[i], response[i], dr[i], drt[i] = trial(drift, boundary, starting_point, ndt, max_t, dt=dt)
        observed[i]=1

    return dict(rt=rt, response=response, observed=observed, dr=dr, drt=drt)

In [ ]:
# sufficient statistics: mean, sd, n
def summary(rt):
    return dict(
        mean = np.mean(rt),
        sd = np.std(rt)
    )

In [ ]:
# creaate the simulator

simulator_graph = bf.make_simulator([context, prior, likelihood, summary])
simulator = bf.make_simulator([context, prior, likelihood])

In [ ]:
df = simulator_graph.sample((1000,))

In [ ]:
f=bf.diagnostics.pairs_samples(
    df,
    variable_keys=["drift", "boundary", "starting_point", "ndt", "mean", "sd"],
    variable_names=[r"$\nu_0$", r"$\nu_1$", r"$\alpha$", "starting point","Ter", "mean RT", "sd RT"]
)


In [ ]:
# create an adapter

adapter = (bf.Adapter()
    .as_set(["rt", "response", "dr", "drt", "observed"]) # dr -> is there double response # drt -> double rt # response -> initial true or false # observed ->
    .constrain(["starting_point", "boundary", "ndt"], lower=0)
    .standardize(include="drift",    mean= 0.7, std=1.2)
    .standardize(include="boundary", mean= 0.5, std=0.7)
    .standardize(include="ndt",   mean=-2.5, std=1.3)
    .concatenate(["drift", "boundary", "ndt","starting_point"], into="inference_variables")
    .concatenate(["rt", "response", "dr", "drt", "observed"], into="summary_variables")
    .rename("n", "inference_conditions")
)

In [ ]:
workflow = bf.BasicWorkflow(
    simulator = simulator,
    adapter = adapter,
    inference_network = bf.networks.CouplingFlow(
        permutation="swap",
        subnet_kwargs=dict(dropout=False)
    ),
    summary_network=bf.networks.DeepSet(
        base_distribution="normal",
        dropout=False
    ),
    inference_variables = ["drift", "boundary", "ndt","starting_point"],
    inference_conditions = ["n"],
    summary_variables = ["rt", "response", "dr", "drt", "observed"]
)

In [ ]:
train_data = simulator.sample(200)
validation_data = simulator.sample(100)

In [ ]:
history=workflow.fit_offline(
    data=train_data,
    epochs=50,
    batch_size=250,
    validation_data=validation_data
)


In [ ]:
test_data = simulator.sample(1_000)
plots=workflow.plot_default_diagnostics(test_data=test_data)

# Feed-Forward Inhibition

In [ ]:
# generation one trial

# the evidence function is implemeted in this function

def FFI_trial(drift, starting_point, boundary, ndt, beta, max_t, max_drt=0.25, s=1, dt=None):
    drift = np.array(drift)
    response = -1
    rt = -1

    if dt is None:
        dt = max_t / 10_000

    t = 0
    evidence = np.random.uniform(0, starting_point, size=len(drift))

    boundary += starting_point  # Adjust boundary based on start
    dr = False  # Initialize double response

    # Initial evidence accumulation
    while all(evidence < boundary) and t < max_t:
        sd = np.random.normal(0,s, size= len(drift))
        for resp, drift_rate in enumerate(drift):
            evidence[resp] += dt * (drift_rate - np.sum(beta * drift[~resp])) + \
                (sd[resp] - beta * np.sum(beta * sd[~resp])) * np.sqrt(dt)
        t += dt

    rt = t + ndt
    drt = 0
    response = np.argmax(evidence)

    # Double response accumulation
    while drt < max_drt:
        sd = np.random.normal(0,s, size= len(drift))
        for resp, drift_rate in enumerate(drift):
            if resp != response:
                evidence[resp] += dt * (drift_rate - np.sum(beta * drift[~resp])) + \
                    (sd[resp] - beta * np.sum(beta * sd[~resp])) * np.sqrt(dt)
                if evidence[resp] >= boundary:
                    dr = True
                    break
        drt += dt

    return rt, response, dr, drt

# Lateral Inhibition

In [ ]:
# generation one trial

# the evidence function is implemeted in this function

def LI_trial(drift, starting_point, boundary, ndt, beta, max_t, max_drt=0.25, s=1, dt=None):
    drift = np.array(drift)
    response = -1
    rt = -1

    if dt is None:
        dt = max_t / 10_000

    t = 0
    evidence = np.random.uniform(0, starting_point, size=len(drift))

    boundary += starting_point  # Adjust boundary based on start
    dr = False  # Initialize double response

    # Initial evidence accumulation
    while all(evidence < boundary) and t < max_t:
        sd = np.random.normal(0,s, size= len(drift))
        for resp, drift_rate in enumerate(drift):
            evidence[resp] += dt * (drift_rate - np.sum(beta * evidence[~resp])) + \
                sd[resp] * np.sqrt(dt)
        t += dt

    rt = t + ndt
    drt = 0
    response = np.argmax(evidence)

    # Double response accumulation
    while drt < max_drt:
        sd = np.random.normal(0,s, size= len(drift))
        for resp, drift_rate in enumerate(drift):
            if resp != response:
                evidence[resp] += dt * (drift_rate - np.sum(beta * evidence[~resp])) + \
                    sd[resp] * np.sqrt(dt)
                if evidence[resp] >= boundary:
                    dr = True
                    break
        drt += dt

    return rt, response, dr, drt

#LCA

- drift = nu = v = drift rate
- boundary= alpha = a = boundary separation
- ndt = tau = Ter = non-decision time
- lamda = leakage rate (decay)
- beta = amount of inhibition
- al_n = number of alternative responses

In [ ]:
# import all needed packages

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import ipynbname
import bayesflow as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from pathlib import Path

In [ ]:
# defing the context

# selecting possible amounts of n -> for our example we could use a fixed value
# of n because all trials are 10000

def context(n=None):
    if n is None:
        n = np.random.randint(2, 251)
    return dict(n=n)


In [ ]:
# defining the priors

# adding lambda (leakage) and beta (inhbition)

def prior(drift=None, boundary=None, starting_point=None, ndt=None, lamda = None, beta = None):
    if drift is None:
        drift=np.random.dirichlet([2, 2])
        drift=np.random.gamma(shape=5, scale=0.5) * drift
    if boundary is None:                    #give it a feasible boundary
        boundary=np.random.gamma(shape=3, scale=1)
    if starting_point is None:
        starting_point=np.random.gamma(shape=7, scale=0.1)
    if lamda is None:
        lamda=np.random.beta(2.5, 15)
    if beta is None:
        beta=np.random.beta(6.5, 25)
    if ndt is None:
        ndt=np.random.exponential(15)
    else:
        ndt=ndt.item()
    return dict(drift=drift, boundary=boundary+starting_point, lamda=lamda, beta=beta, starting_point=starting_point, ndt=ndt)



In [ ]:

def LCA_trial(drift, starting_point, boundary, ndt, beta, lamda, max_t, max_drt=0.25, s=1, dt=None):
    drift = np.array(drift)
    response = -1
    rt = -1

    if dt is None:
        dt = max_t / 10_000

    t = 0
    evidence = np.random.uniform(0, starting_point, size=len(drift))

    boundary += starting_point  # Adjust boundary based on start
    dr = False  # Initialize double response


    # Initial evidence accumulation
    while all(evidence < boundary) and t < max_t:
        for resp, drift_rate in enumerate(drift):
            inhibition_effect = beta * (np.sum(evidence) - evidence[resp])
            evidence[resp] += dt * (drift_rate - lamda * evidence[resp] - inhibition_effect) + np.random.normal(0, np.sqrt(s**2 * dt))
            if evidence[resp] < 0:
              evidence[resp] = 0

        t += dt

    rt = t + ndt
    drt = 0
    response = np.where(evidence > boundary)[0]  # Nimm nur das erste Element des Tuples

    if response.size == 1:
        response = response.item()  # Konvertiere in eine Zahl
    else:
        response = -1  # Falls leer oder mehrere Werte, setze Standardwert

    # Double response accumulation
    while drt < max_drt and not dr:  # Stoppe, wenn dr True wird
        for resp, drift_rate in enumerate(drift):
            if response != -1 and resp != response:
                inhibition_effect = beta * (np.sum(evidence) - evidence[resp])
                evidence[resp] += dt * (drift_rate - lamda * evidence[resp] - inhibition_effect) + np.random.normal(0, np.sqrt(s**2 * dt))
                if evidence[resp] < 0:
                  evidence[resp] = 0
                if evidence[resp] >= boundary:
                    dr = True
                    break
        drt += dt  # `drt` nur erhöhen, solange dr False ist


    return rt, response, dr, drt



In [ ]:
# generate data for n trials
# keep the data.shape always to max_n
# the rest is filled with 0s


# insert dt and drt as a liklyhood

def likelihood(n, drift, boundary, starting_point, ndt, beta, lamda, max_t=3.0, dt=0.02, max_n=300):
    rt = np.zeros(max_n)
    response = np.zeros(max_n)
    observed = np.zeros(max_n)
    dr=np.zeros(max_n)
    drt=np.zeros(max_n)



    for i in range(n):
        rt[i], response[i], dr[i], drt[i], = trial(drift, boundary, starting_point, lamda, beta, ndt, max_t, dt=dt)
        observed[i]=1

    return dict(rt=rt, response=response, observed=observed, dr=dr, drt=drt)

In [ ]:
# sufficient statistics: mean, sd, n
def summary(rt):
    return dict(
        mean = np.mean(rt),
        sd = np.std(rt)
    )

In [ ]:
# creaate the simulator

simulator_graph = bf.make_simulator([context, prior, likelihood, summary])
simulator = bf.make_simulator([context, prior, likelihood])

In [ ]:
df = simulator_graph.sample(1000)

In [ ]:
f=bf.diagnostics.pairs_samples(
    df,
    variable_keys=["drift", "boundary", "starting_point", "ndt", "beta", "lamda", "mean", "sd"],
    variable_names=[r"$\nu_0$", r"$\nu_1$", r"$\alpha$", "starting point","Ter","beta", "lamdba", "mean RT", "sd RT"]
)

In [ ]:
# create an adapter

adapter = (bf.Adapter()
    .as_set(["rt", "response", "dr", "drt", "observed"])
    .constrain(["starting_point", "boundary", "ndt","beta", "lamda"], lower=0)
    .standardize(include="drift",    mean= 0.7, std=1.2)
    .standardize(include="boundary", mean= 0.5, std=0.7)
    .standardize(include="ndt",   mean=-2.5, std=1.3)
    .concatenate(["drift", "boundary", "ndt","starting_point","beta", "lamda"], into="inference_variables")
    .concatenate(["rt", "response", "dr", "drt", "observed"], into="summary_variables")
    .rename("n", "inference_conditions")
)

In [ ]:
workflow = bf.BasicWorkflow(
    simulator = simulator,
    adapter = adapter,
    inference_network = bf.networks.CouplingFlow(
        permutation="swap",
        subnet_kwargs=dict(dropout=False)
    ),
    summary_network=bf.networks.DeepSet(
        base_distribution="normal",
        dropout=False
    ),
    inference_variables = ["drift", "boundary", "ndt","starting_point", "beta", "lamda"],
    inference_conditions = ["n"],
    summary_variables = ["rt", "response", "dr", "drt", "observed"]
)

In [ ]:
train_data = simulator.sample(5000)
validation_data = simulator.sample(1000)

In [ ]:
history=workflow.fit_offline(
    data=train_data,
    epochs=50,
    batch_size=250,
    validation_data=validation_data
)

In [ ]:
test_data = simulator.sample(1_000)
plots=workflow.plot_default_diagnostics(test_data=test_data)

# Real data application


In [ ]:
data_inference = pd.read_csv("s1.csv")

In [ ]:
######################################################
#               INFERENCE ON REAL DATA               #
######################################################

data_inference = pd.read_csv("s1.csv")

data_inference_grouped = data_inference.groupby(["subject", "condition"])

data_inference_dict = {
    key: np.array([group[key].values.reshape(10000, 1) for _, group in data_inference_grouped])
             for key in ['rt', 'response', 'observed']}

data_inference_dict["n"] = np.sum(data_inference_dict["observed"], axis=1)

posterior_samples = workflow.sample(conditions=data_inference_dict, num_samples=1_000)

posterior = {key: value[0] for key, value in posterior_samples.items()}
data = data_inference_grouped.get_group(('s1', 'speed'))



In [ ]:
# PLOTS
# paired plots
f=bf.diagnostics.pairs_posterior(estimates=posterior) # should return paired plots


# ecdf plot
def ecdf(rt, response, observed, **kwargs):
    observed_mask = (observed == 1)
    response_0_mask = ((response == 0) & observed_mask)
    response_1_mask = ((response == 1) & observed_mask)

    response_0_prop = np.sum(response_0_mask) / np.sum(observed_mask)
    response_1_prop = np.sum(response_1_mask) / np.sum(observed_mask)

    response_0_ecdf = stats.ecdf(rt[response_0_mask]).cdf
    response_0_ecdf = response_0_prop * response_0_ecdf.evaluate(np.linspace(0, 1, 101))

    response_1_ecdf = stats.ecdf(rt[response_1_mask]).cdf
    response_1_ecdf = response_1_prop * response_1_ecdf.evaluate(np.linspace(0, 1, 101))

    return response_0_ecdf, response_1_ecdf

plot_data = ecdf(**data)


# some other plot (line plot i think)
posterior_predictives = simulator.sample(1000, **posterior)

plot_data_predictive = []
for i in range(1000):
    x = { key: value[i:i+1,...] for key, value in posterior_predictives.items()}
    plot_data_predictive.append(ecdf(**x))
plot_data_predictive = np.array(plot_data_predictive)


plot_data_quantiles = np.quantile(
    plot_data_predictive,
    q = [0.25, 0.5, 0.75],
    axis=0
)
plot_data_quantiles.shape

t = np.linspace(0, 1, 101)
cols = ["red", "blue"]
for i, lab in enumerate(["Incorrect", "Correct"]):
    plt.plot(t, plot_data[i], label=lab, color=cols[i])
    plt.plot(t, plot_data_quantiles[1, i, :], color=cols[i], alpha=0.5, label="median predictive")
    plt.fill_between(
        t,
        plot_data_quantiles[0,  i,:],
        plot_data_quantiles[-1, i,:],
        label="50% predictive interval",
        color=cols[i],
        alpha=0.3
    )
f=plt.legend()

In [ ]:
data_inference = pd.read_csv("s1.csv")
data_inference['condition'].unique()
data_inference_grouped = data_inference.groupby(["subject", "condition"])
data_inference_dict = {
    key: np.array([group[key].values.reshape(10000, 1) for _, group in data_inference_grouped])
             for key in ['rt', 'response','dr', 'drt', 'observed']}
data_inference_dict["n"] = np.sum(data_inference_dict["observed"], axis=1)
print({key: value.shape for key, value in data_inference_dict.items()})
posterior_samples = workflow.sample(conditions=data_inference_dict, num_samples=1_000)
# pick the first participant, first condition
posterior = {key: value[0] for key, value in posterior_samples.items()}
data = data_inference_grouped.get_group(('s1', 'speed'))
f=bf.diagnostics.pairs_posterior(estimates=posterior)
def ecdf(rt, response, observed, **kwargs):
    observed_mask = (observed == 1)
    response_0_mask = ((response == 0) & observed_mask)
    response_1_mask = ((response == 1) & observed_mask)

    response_0_prop = np.sum(response_0_mask) / np.sum(observed_mask)
    response_1_prop = np.sum(response_1_mask) / np.sum(observed_mask)

    response_0_ecdf = stats.ecdf(rt[response_0_mask]).cdf
    response_0_ecdf = response_0_prop * response_0_ecdf.evaluate(np.linspace(0, 1, 101))

    response_1_ecdf = stats.ecdf(rt[response_1_mask]).cdf
    response_1_ecdf = response_1_prop * response_1_ecdf.evaluate(np.linspace(0, 1, 101))

    return response_0_ecdf, response_1_ecdf

plot_data = ecdf(**data)
posterior_predictives = simulator.sample(1000, **posterior)
plot_data_predictive = []
for i in range(1000):
    x = { key: value[i:i+1,...] for key, value in posterior_predictives.items()}
    plot_data_predictive.append(ecdf(**x))
plot_data_predictive = np.array(plot_data_predictive)
plot_data_quantiles = np.quantile(
    plot_data_predictive,
    q = [0.25, 0.5, 0.75],
    axis=0
)
plot_data_quantiles.shape
t = np.linspace(0, 2, 101)
cols = ["red", "blue"]
for i, lab in enumerate(["Incorrect", "Correct"]):
    plt.plot(t, plot_data[i], label=lab, color=cols[i])
    plt.plot(t, plot_data_quantiles[1, i, :], color=cols[i], alpha=0.5, label="median predictive")
    plt.fill_between(
        t,
        plot_data_quantiles[0,  i,:],
        plot_data_quantiles[-1, i,:],
        label="50% predictive interval",
        color=cols[i],
        alpha=0.3
    )
f=plt.legend()

In [ ]:
data_inference = pd.read_csv("s1.csv")

In [ ]:
data_inference['condition'].unique()

In [ ]:
data_inference_grouped = data_inference.groupby(["subject", "condition"])

In [ ]:
data_inference_dict = {
    key: np.array([group[key].values.reshape(10000, 1) for _, group in data_inference_grouped])
             for key in ['rt', 'response','dr', 'drt', 'observed']}
data_inference_dict["n"] = np.sum(data_inference_dict["observed"], axis=1)
print({key: value.shape for key, value in data_inference_dict.items()})

In [ ]:
posterior_samples = workflow.sample(conditions=data_inference_dict, num_samples=1000)

In [ ]:
# pick the first participant, first condition
posterior = {key: value[0] for key, value in posterior_samples.items()}
posterior['drift'].shape

In [ ]:
priors = dict(drift=test_data["drift"], boundary=test_data["boundary"], ndt=test_data["ndt"], starting_point=test_data["starting_point"])
f=bf.diagnostics.pairs_posterior(estimates=posterior)

In [ ]:
def ecdf(rt, response, observed, **kwargs):
    observed_mask = (observed == 1)
    response_0_mask = ((response == 0) & observed_mask)
    response_1_mask = ((response == 1) & observed_mask)

    response_0_prop = np.sum(response_0_mask) / np.sum(observed_mask)
    response_1_prop = np.sum(response_1_mask) / np.sum(observed_mask)

    response_0_ecdf = stats.ecdf(rt[response_0_mask]).cdf
    response_0_ecdf = response_0_prop * response_0_ecdf.evaluate(np.linspace(0, 1, 101))

    response_1_ecdf = stats.ecdf(rt[response_1_mask]).cdf
    response_1_ecdf = response_1_prop * response_1_ecdf.evaluate(np.linspace(0, 1, 101))

    return response_0_ecdf, response_1_ecdf

In [ ]:
data = data_inference_grouped.get_group(('s1', 'speed'))
plot_data = ecdf(**data)

In [ ]:
posterior_predictives = simulator.sample(batch_shape=1000, posterior=posterior)

In [ ]:
plot_data_predictive = []
for i in range(1000):
    x = { key: value[i:i+1,...] for key, value in posterior_predictives.items()}
    plot_data_predictive.append(ecdf(**x))

plot_data_predictive = np.array(plot_data_predictive)

In [ ]:
# Compute quantiles
plot_data_quantiles = np.quantile(plot_data_predictive, q=[0.25, 0.5, 0.75], axis=0)
plot_data_quantiles.shape

In [ ]:
# Plot results
t = np.linspace(0, 2, 101)
cols = ["red", "blue"]
for i, lab in enumerate(["Incorrect", "Correct"]):
    plt.plot(t, plot_data[i], label=lab, color=cols[i])
    plt.plot(t, plot_data_quantiles[1, i, :], color=cols[i], alpha=0.5, label="median predictive")
    plt.fill_between(t, plot_data_quantiles[0, i, :], plot_data_quantiles[-1, i, :],
                     label="50% predictive interval", color=cols[i], alpha=0.3)

f = plt.legend()